<a href="https://colab.research.google.com/github/gmayad-jpg/senalesysistemas2025-2/blob/main/Parcial2SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Parcial 2 Señales y Sistemas**
#**Geronimo Maya Diaz**


#Instalación de Librerías

#Página principal

In [1]:
!pip install streamlit -q

In [2]:
!pip install streamlit numpy scipy matplotlib yt-dlp pydub

#Crear carpeta pages para trabajar Multiapp Streamlit

In [3]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


#Página principal

In [4]:
%%writefile 0_👋_Hello.py

import streamlit as st

st.set_page_config(
    page_title="Parcial 2 Señales y Sistemas",
    page_icon="🔬", # Icono relacionado con la ciencia
)

# Usando columnas para una mejor distribución
col1, col2, col3 = st.columns([1, 4, 1]) # Columnas laterales más estrechas

with col2: # Contenido principal en la columna central
    st.markdown("<h1 style='text-align: center; color: #1E90FF;'> Bienvenido al Parcial 2 de Señales y Sistemas </h1>", unsafe_allow_html=True) # Título centrado y con estilo
    st.markdown("---") # Separador

    st.markdown("""
    ¡Hola! 👋 Este dashboard interactivo ha sido creado como parte del Parcial 2 de la asignatura Señales y Sistemas. Aquí podrás explorar simulaciones y análisis de conceptos clave de la materia.
    """)

    # Aplicar estilo azul a los subtítulos
    st.markdown("<h3 style='color: #1E90FF;'>Presentado por:</h3>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: #1E90FF;'>Geronimo Maya Diaz</h3>", unsafe_allow_html=True) # Nombre con formato de encabezado


    st.markdown("---") # Otro separador



st.sidebar.success("Seleccciona una pregunta a explorar.")

Overwriting 0_👋_Hello.py


##Punto 1

In [5]:
%%writefile 1_Punto_1.py
import streamlit as st
import numpy as np
from scipy.fft import fft, ifft, fftshift, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
import yt_dlp
import os
import subprocess
import time
import scipy.signal as signal
from io import BytesIO

# --- Configuración de la página de Streamlit ---
st.set_page_config(
    page_title="Modulador DSB-SC",
    page_icon="🎛️",
    layout="wide",
)

st.title("🎛️ Dashboard de Modulación DSB-SC")
st.write(
    "Este panel permite simular la **Modulación de Doble Banda Lateral con Portadora Suprimida (DSB-SC)** "
    "y comparar dos métodos de demodulación coherente: "
    "filtrado ideal en frecuencia (FFT) y filtrado práctico en tiempo (filtros IIR)."
)

st.markdown("---")

st.markdown("""
### Modulación de Doble Banda Lateral con Portadora Suprimida (DSB-SC)
La DSB-SC es una técnica de modulación que transmite ambas bandas laterales del mensaje, pero suprime la portadora, ahorrando potencia en comparación con la AM tradicional.

**Modelo Matemático del Proceso de Modulación y Demodulación DSB-SC**

La señal modulada es:
$$ s_{DSB-SC}(t) = m(t) c(t) = m(t) A_c \\cos(2\\pi f_c t) $$

En el dominio de la frecuencia:
$$ S_{DSB-SC}(f) = \\frac{A_c}{2} [M(f - f_c) + M(f + f_c)] $$
""")

st.markdown("---")

# --- Funciones Auxiliares ---

@st.cache_data
def load_audio_from_youtube(url, duration_s=5, start_s=0):
    """
    Descarga un segmento de audio de YouTube, lo convierte a WAV (44.1 kHz, mono)
    y lo carga como un arreglo numpy.
    """
    temp_audio_file_base = 'temp_audio'
    cropped_wav_file = 'cropped_audio.wav'

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': temp_audio_file_base,
        'quiet': True,
        'noprogress': True,
        'logtostderr': False
    }

    try:
        # Limpiar archivos temporales previos
        for ext in ['webm', 'm4a', 'mp3', 'wav']:
            file_to_check = temp_audio_file_base + '.' + ext
            if os.path.exists(file_to_check):
                os.remove(file_to_check)
        if os.path.exists(cropped_wav_file): os.remove(cropped_wav_file)

        # Descargar audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            downloaded_file = ydl.prepare_filename(info)

        # Convertir y recortar con ffmpeg
        command = [
            'ffmpeg', '-i', str(downloaded_file), '-ss', str(start_s),
            '-t', str(duration_s), '-f', 'wav', '-acodec', 'pcm_s16le',
            '-ar', '44100', '-ac', '1', str(cropped_wav_file), '-y'
        ]
        subprocess.run(command, check=True, capture_output=True, text=True)

        # Leer el archivo WAV resultante
        fs_audio, audio_data = wavfile.read(cropped_wav_file)

        # Normalizar datos de audio a [-1, 1]
        if np.max(np.abs(audio_data)) > 0:
            audio_data = audio_data.astype(np.float64) / np.max(np.abs(audio_data))
        else:
            audio_data = audio_data.astype(np.float64)

        # Crear vector de tiempo
        t = np.linspace(0, len(audio_data) / fs_audio, len(audio_data), endpoint=False)

        # Limpiar archivos descargados
        if os.path.exists(downloaded_file): os.remove(downloaded_file)
        if os.path.exists(cropped_wav_file): os.remove(cropped_wav_file)

        return audio_data, fs_audio, t
    except Exception as e:
        st.error(f"Error procesando audio: {e}. Asegúrese de que 'ffmpeg' esté instalado y que la URL sea válida.")
        return None, None, None

def plot_signal_time(t, sig, title):
    """
    Grafica una señal en el dominio del tiempo.
    """
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, sig, lw=1)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True)
    ax.margins(x=0.01, y=0.1)
    plt.tight_layout()
    return fig

def plot_signal_freq(sig, fs, title, xlim_freq=None):
    """
    Calcula y grafica el espectro de magnitud de una señal usando FFT.
    """
    fig, ax = plt.subplots(figsize=(10, 4))
    N = len(sig)
    if N == 0: return fig
    yf = fftshift(fft(sig))
    xf = fftshift(fftfreq(N, 1 / fs))
    ax.plot(xf, np.abs(yf) / N, lw=1)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Frecuencia (Hz)")
    ax.set_ylabel("Magnitud")
    ax.grid(True)
    if xlim_freq: ax.set_xlim(xlim_freq)
    plt.tight_layout()
    return fig

def plot_filter_characteristics(filter_system, fs, title_prefix, plot_type="Polar"):
    """
    Grafica la respuesta en frecuencia (Bode) y el diagrama de polos y ceros de un filtro.
    """
    # Diagrama de Bode
    fig_bode, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    w, h = signal.freqz(filter_system.num, filter_system.den, worN=8192, fs=fs)
    ax_mag.semilogx(w, 20 * np.log10(np.abs(h)))
    ax_mag.set_title(f"Respuesta en Magnitud - {title_prefix}")
    ax_mag.set_ylabel("Magnitud (dB)")
    ax_mag.grid(True, which='both')

    ax_phase.semilogx(w, np.unwrap(np.angle(h)) * 180 / np.pi)
    ax_phase.set_title(f"Respuesta en Fase - {title_prefix}")
    ax_phase.set_xlabel("Frecuencia (Hz)")
    ax_phase.set_ylabel("Fase (grados)")
    ax_phase.grid(True, which='both')
    plt.tight_layout()

    # Diagrama de Polos y Ceros
    fig_pz, ax_pz = plt.subplots(figsize=(6, 6))
    poles, zeros, _ = signal.tf2zpk(filter_system.num, filter_system.den)
    ax_pz.scatter(np.real(poles), np.imag(poles), marker='x', color='r', s=100, label='Polos')
    if zeros.size > 0:
        ax_pz.scatter(np.real(zeros), np.imag(zeros), marker='o', color='b', s=100, facecolors='none', label='Ceros')

    if plot_type == "Polar":
        theta = np.linspace(0, 2*np.pi, 100)
        ax_pz.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.5, label='Círculo Unitario')

    ax_pz.set_title(f"Polos y Ceros - {title_prefix}")
    ax_pz.set_xlabel("Eje Real")
    ax_pz.set_ylabel("Eje Imaginario")
    ax_pz.grid(True)
    ax_pz.legend()
    ax_pz.set_aspect('equal', adjustable='box')

    return fig_bode, fig_pz


# --- FUNCIÓN DE MODULACIÓN DSB-SC (ÚNICA) ---

def modulate_dsbsc(m_t, t, fs, fc):
    """
    Realiza la modulación DSB-SC mediante la multiplicación de la señal mensaje por la portadora.
    """
    s_dsbsc_t = m_t * (2 * np.cos(2 * np.pi * fc * t))
    return s_dsbsc_t, None

# --- FUNCIONES DE DEMODULACIÓN DSB-SC ---

def demodulate_dsbsc_fft(s_dsbsc_t, t, fs, fc, message_bw):
    """
    Realiza la demodulación DSB-SC usando filtrado ideal (LPF) en el dominio de la frecuencia (FFT).
    """
    # Multiplicar por portadora coherente
    v_t = s_dsbsc_t * (2 * np.cos(2 * np.pi * fc * t))
    V_f = fft(v_t)
    freqs = fftfreq(len(t), 1 / fs)

    # Crear máscara de filtro paso-bajo ideal para recuperar el mensaje
    cutoff_freq = message_bw * 1.2
    mask = np.zeros_like(freqs)
    mask[np.abs(freqs) <= cutoff_freq] = 1

    # Aplicar filtro paso-bajo ideal
    M_demod_f = V_f * mask
    m_demod_t = np.real(ifft(M_demod_f))

    # Crear un objeto de filtro dummy para compatibilidad
    b, a = signal.butter(5, cutoff_freq / (fs / 2), btype='low', analog=False)
    demod_filter_system = signal.TransferFunction(b, a, dt=1/fs)

    return m_demod_t, demod_filter_system

def demodulate_dsbsc_iir(s_dsbsc_t, t, fs, fc, message_bw, order):
    """
    Realiza la demodulación DSB-SC usando un filtro paso-bajo IIR en el dominio del tiempo.
    """
    # Multiplicar por portadora coherente
    v_t = s_dsbsc_t * (2 * np.cos(2 * np.pi * fc * t))
    nyquist = 0.5 * fs
    cutoff_freq = message_bw * 1.2

    if cutoff_freq >= nyquist: cutoff_freq = nyquist - 1
    if cutoff_freq <= 0: cutoff_freq = 1

    # Diseñar filtro Butterworth paso-bajo IIR
    Wn = cutoff_freq / nyquist
    b, a = signal.butter(order, Wn, btype='low', analog=False)

    # Aplicar filtro paso-bajo a la señal v(t) (filtfilt para fase cero)
    m_demod_t = signal.filtfilt(b, a, v_t)
    demod_filter_system = signal.TransferFunction(b, a, dt=1/fs)

    return m_demod_t, demod_filter_system


# --- Panel de Control en la Barra Lateral ---
st.sidebar.header("⚙️ Parámetros de Simulación")

# 1. Selección de método (para la demodulación LPF)
implementation_method = st.sidebar.selectbox(
    "1. Seleccione el Método de Demodulación",
    ("Filtrado en Frecuencia (Ideal)", "Filtrado en Tiempo (Práctico)"),
    help="Elija entre un Filtro Paso-Bajo teórico con FFT o uno práctico con filtros IIR para la demodulación."
)

# 2. Parámetros del filtro (si es práctico)
filter_order = 1
if implementation_method == "Filtrado en Tiempo (Práctico)":
    filter_order = st.sidebar.slider(
        "Orden del Filtro IIR (LPF de Demodulación)", 2, 20, 8, 1,
        help="Un orden mayor crea un Filtro Paso-Bajo (LPF) más abrupto y mejora la separación entre el mensaje y la componente de 2*fc."
    )

st.sidebar.markdown("---")

# 3. Selección de señal
msg_type = st.sidebar.selectbox(
    "2. Seleccione la Señal Mensaje",
    ("Audio de YouTube", "Pulso Rectangular"),
    help="Elija la señal de información que desea modular."
)

m_t, fs, t, message_bw = None, 44100, None, 0
duration_s = st.sidebar.slider("Duración (s)", 1, 20, 5, 1)

if msg_type == "Pulso Rectangular":
    # Generar un pulso rectangular
    t = np.linspace(0, duration_s, int(duration_s * fs), endpoint=False)
    pulse_width = st.sidebar.slider("Ancho del Pulso (s)", 0.1, 2.0, 0.5, 0.1)
    m_t = np.zeros_like(t)
    m_t[t < pulse_width] = 1.0
    message_bw = 2 / pulse_width if pulse_width > 0 else 0
    st.sidebar.info(f"Ancho de banda estimado: {message_bw:.1f} Hz")
else: # Audio de YouTube
    youtube_url = st.sidebar.text_input("URL de YouTube", "https://youtu.be/Lf566pMNCOw?si=rJXEGvka2NdDduzS")
    start_time_s = st.sidebar.slider("Tiempo de inicio (s)", 0, 300, 50, 1)
    if youtube_url:
        # Cargar audio de YouTube
        with st.spinner(f"Cargando {duration_s}s de audio desde el segundo {start_time_s}..."):
            m_t, fs, t = load_audio_from_youtube(youtube_url, duration_s=duration_s, start_s=start_time_s)
        message_bw = 10000
        if fs:
            st.sidebar.info(f"Ancho de banda asumido: {message_bw} Hz\nFrec. Muestreo: {fs} Hz")

# Parámetro de frecuencia de portadora
fc_max = (fs / 2) - (message_bw * 1.1) if fs and message_bw > 0 else 20000
fc = st.sidebar.slider("3. Frecuencia de Portadora (fc) [Hz]", 1000.0, fc_max, min(10000.0, fc_max), 500.0)

# --- Lógica Principal de la Aplicación ---
if m_t is not None and t is not None and fs is not None:
    st.header("1. Señal Mensaje Original (m(t))")
    col1, col2 = st.columns(2)
    with col1:
        st.pyplot(plot_signal_time(t, m_t, "Mensaje Original en Tiempo"))
    with col2:
        st.pyplot(plot_signal_freq(m_t, fs, "Espectro del Mensaje Original", xlim_freq=(-message_bw * 1.5, message_bw * 1.5)))

    if msg_type == "Audio de YouTube":
        buf = BytesIO()
        wavfile.write(buf, fs, np.int16(m_t / np.max(np.abs(m_t)) * 32767))
        st.audio(buf, format="audio/wav")

    st.markdown("---")
    st.header(f"2. Proceso de Modulación DSB-SC")

    # MODULACIÓN
    s_dsbsc_t, mod_filter = modulate_dsbsc(m_t, t, fs, fc)

    if s_dsbsc_t is not None:
        st.pyplot(plot_signal_freq(s_dsbsc_t, fs, f"Espectro Modulado DSB-SC", xlim_freq=(-fc - message_bw * 2, fc + message_bw * 2)))

    st.markdown("---")
    st.header(f"3. Proceso de Demodulación ({implementation_method})")

    # DEMODULACIÓN
    if implementation_method == "Filtrado en Frecuencia (Ideal)":
        m_demod_t, demod_filter = demodulate_dsbsc_fft(s_dsbsc_t, t, fs, fc, message_bw)
    else:
        m_demod_t, demod_filter = demodulate_dsbsc_iir(s_dsbsc_t, t, fs, fc, message_bw, filter_order)

    st.subheader("Señal Final Recuperada")
    col3, col4 = st.columns(2)
    with col3:
        if m_demod_t is not None:
            st.pyplot(plot_signal_time(t, m_demod_t, "Mensaje Recuperado en Tiempo"))
    with col4:
        if m_demod_t is not None:
            st.pyplot(plot_signal_freq(m_demod_t, fs, "Espectro del Mensaje Recuperado", xlim_freq=(-message_bw * 1.5, message_bw * 1.5)))

    if msg_type == "Audio de YouTube" and m_demod_t is not None:
        # Reproducir audio demodulado
        buf = BytesIO()
        max_val = np.max(np.abs(m_demod_t))
        if max_val > 1e-6:
             wavfile.write(buf, fs, np.int16(m_demod_t / max_val * 32767))
        else:
             wavfile.write(buf, fs, np.zeros_like(m_demod_t, dtype=np.int16))

        st.audio(buf, format="audio/wav")


    st.markdown("---")
    st.header("4. Análisis del Filtro Paso-Bajo (LPF) de Demodulación")

    plot_type_param = "Polar"

    # Mostrar análisis de filtro de demodulación (Demod LPF)
    if demod_filter is not None:
        with st.expander("Ver Diagrama de Bode y Polos/Ceros del LPF"):
            fig_bode, fig_pz = plot_filter_characteristics(demod_filter, fs, "LPF de Demodulación", plot_type_param)
            st.pyplot(fig_bode)
            st.pyplot(fig_pz)
else:
    st.info("Configure los parámetros en la barra lateral para iniciar la simulación.")

Writing 1_Punto_1.py


In [6]:
!mv 1_Punto_1.py pages/

##Punto 2

In [7]:
%%writefile 2_Punto_2.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

# --- Configuración de la página ---
st.set_page_config(
    page_title="Análisis de Sistemas de 2do Orden",
    page_icon="🔬",
    layout="wide",
)

st.markdown("<h1 style='color: #1E90FF;'>🔬 Dashboard de Simulación para Sistemas de Segundo Orden</h1>", unsafe_allow_html=True)
st.write(
    "Este panel permite simular sistemas de segundo orden, como el modelo masa-resorte-amortiguador "
    "y su circuito eléctrico análogo. "
)

st.markdown("---") # Separador

st.markdown("""
### Sistemas de Segundo Orden

Los sistemas de segundo orden son fundamentales en el análisis de sistemas dinámicos en diversas áreas de la ingeniería, como el control, la mecánica y la electrónica. Se caracterizan por tener una ecuación diferencial de segundo orden que describe su comportamiento. La forma canónica de la función de transferencia de un sistema de segundo orden es:

$$ G(s) = \\frac{\\omega_n^2}{s^2 + 2\\zeta\\omega_n s + \\omega_n^2} $$

Donde:
*   $ \\omega_n $ es la **frecuencia natural no amortiguada** del sistema, que representa la frecuencia de oscilación si no hubiera amortiguamiento.
*   $ \\zeta $ es el **factor de amortiguamiento**, un parámetro adimensional que determina cómo se disipa la energía en el sistema. El valor de $ \\zeta $ define el tipo de respuesta del sistema:
    *   $ \\zeta < 0 $: **Inestable** (la respuesta crece exponencialmente).
    *   $ \\zeta = 0 $: **No amortiguado** (oscilaciones continuas sin decaer).
    *   $ 0 < \\zeta < 1 $: **Subamortiguado** (la respuesta oscila con amplitud decreciente hasta estabilizarse).
    *   $ \\zeta = 1 $: **Críticamente amortiguado** (la respuesta alcanza el estado estacionario lo más rápido posible sin sobrepasar el valor final).
    *   $ \\zeta > 1 $: **Sobreamortiguado** (la respuesta se acerca al estado estacionario lentamente sin oscilaciones).

Estos sistemas son análogos a circuitos RLC (Resistencia-Inductancia-Capacitancia) en serie o paralelo y a sistemas mecánicos masa-resorte-amortiguador, donde los componentes (R, L, C o m, k, c) determinan los valores de $ \\omega_n $ y $ \\zeta $.
""")

st.markdown("---") # Separador

# --- Barra lateral para controles de usuario ---
st.sidebar.header("⚙️ Parámetros de Simulación")

# Selección del tipo de respuesta
response_type = st.sidebar.selectbox(
    "1. Seleccione el Tipo de Respuesta",
    (
        "Subamortiguada",
        "Críticamente Amortiguada",
        "Sobreamortiguada",
        "Inestable",
    ),
    help="Define el comportamiento general del sistema. [cite: 28]",
)

# Ajuste del factor de amortiguamiento (zeta) y frecuencia natural (omega_n)
if response_type == "Subamortiguada":
    zeta = st.sidebar.slider(
        "Factor de Amortiguamiento (ζ)", 0.01, 0.99, 0.3, 0.01
    )
elif response_type == "Críticamente Amortiguada":
    zeta = 1.0
    st.sidebar.info("Para amortiguamiento crítico, ζ = 1.")
elif response_type == "Sobreamortiguada":
    zeta = st.sidebar.slider("Factor de Amortiguamiento (ζ)", 1.1, 5.0, 1.5, 0.1)
else:  # Instable
    zeta = st.sidebar.slider("Factor de Amortiguamiento (ζ)", -1.0, -0.01, -0.5, 0.01)

omega_n = st.sidebar.slider(
    "Frecuencia Natural (ωn) [rad/s]", 1.0, 20.0, 5.0, 0.5,
    help="Frecuencia a la que el sistema oscilaría sin amortiguamiento."
)

st.sidebar.markdown("---")
st.sidebar.info(
    "**Fuente:** Parcial 2 SyS 2025-1\n\n"
    "**Profesor:** Andrés Marino Álvarez Meza, Ph.D. "
)


# --- Lógica de Simulación y Cálculos ---

# 1. Definición de las funciones de transferencia
# El sistema físico (planta) se modela como el sistema en Lazo Abierto.
# Su forma canónica es: wn^2 / (s^2 + 2*zeta*wn*s + wn^2)
num_ol = [omega_n ** 2]
den_ol = [1, 2 * zeta * omega_n, omega_n ** 2]
sys_ol = signal.TransferFunction(num_ol, den_ol)

# El sistema en Lazo Cerrado se obtiene aplicando realimentación unitaria al de lazo abierto.
# G_cl(s) = G_ol(s) / (1 + G_ol(s))
# El numerador sigue siendo el mismo.
num_cl = num_ol
# El denominador es Den_ol(s) + Num_ol(s)
den_cl = [den_ol[0], den_ol[1], den_ol[2] + num_ol[0]]
sys_cl = signal.TransferFunction(num_cl, den_cl)

# 2. Estimación de componentes de los sistemas [cite: 30]
# Sistema Mecánico (asumiendo m=1 kg)
m = 1.0
c = 2 * zeta * omega_n * m
k = (omega_n ** 2) * m

# Sistema Eléctrico (asumiendo C=1 F para tener una solución única)
C_elec = 1.0
# A partir de wn^2 = 1/(LC) y 2*zeta*wn = 1/(RC)
if zeta != 0 and omega_n != 0:
    R_elec = 1 / (2 * zeta * omega_n * C_elec) if (zeta * omega_n * C_elec) != 0 else float('inf')
    L_elec = 1 / ((omega_n ** 2) * C_elec) if (omega_n ** 2 * C_elec) != 0 else float('inf')
else:
    R_elec, L_elec = float('inf'), float('inf')


# 3. Cálculo de parámetros temporales (para sistema en lazo cerrado)
ts, tp, mp, tr = None, None, None, None

# Caso 1: Subamortiguado (0 < ζ < 1) -> Usamos fórmulas clásicas
if 0 < zeta < 1:
    # Tiempo de establecimiento (Ts) al 2%
    ts = 4 / (zeta * omega_n)
    # Tiempo de pico (Tp)
    tp = np.pi / (omega_n * np.sqrt(1 - zeta ** 2))
    # Sobreimpulso máximo (Mp)
    mp = 100 * np.exp((-zeta * np.pi) / np.sqrt(1 - zeta ** 2))

    # Cálculo numérico del tiempo de levantamiento (Tr)
    t_step_calc, y_step_calc = signal.step(sys_cl)
    try:
        final_value = y_step_calc[-1]
        tr_10 = t_step_calc[np.where(y_step_calc >= 0.1 * final_value)[0][0]]
        tr_90 = t_step_calc[np.where(y_step_calc >= 0.9 * final_value)[0][0]]
        tr = tr_90 - tr_10
    except IndexError:
        tr = "No calculable"


# Caso 2: Inestable (ζ < 0) -> Los parámetros no aplican
elif zeta < 0:
    ts = "No aplica (Inestable)"
    tr = "No aplica (Inestable)"
    mp = "No aplica (Inestable)"
    tp = "No aplica (Inestable)"

# Caso 3: Críticamente amortiguado o Sobreamortiguado (ζ >= 1) -> Cálculo numérico
else:
    mp = 0.0  # No hay sobreimpulso
    tp = "No aplica"
    t_step_calc, y_step_calc = signal.step(sys_cl)
    final_value = y_step_calc[-1]

    if final_value > 1e-6: # Evitar división por cero si la respuesta es nula
        try:
            # Tiempo de establecimiento (Ts): último momento fuera de la banda del 2%
            settling_indices = np.where(np.abs(y_step_calc - final_value) >= 0.02 * final_value)[0]
            ts = t_step_calc[settling_indices[-1]] if settling_indices.size > 0 else 0.0

            # Tiempo de levantamiento (Tr): 10% a 90%
            tr_10_idx = np.where(y_step_calc >= 0.1 * final_value)[0]
            tr_90_idx = np.where(y_step_calc >= 0.9 * final_value)[0]
            tr = t_step_calc[tr_90_idx[0]] - t_step_calc[tr_10_idx[0]] if tr_10_idx.size > 0 and tr_90_idx.size > 0 else "No calculable"
        except Exception:
            ts, tr = "Error numérico", "Error numérico"
    else:
        ts, tr = "N/A", "N/A"

# --- Visualización de Resultados ---
col1, col2 = st.columns(2)

with col1:
    st.markdown("<h3 style='color: #1E90FF;'>Componentes Estimados del Sistema</h3>", unsafe_allow_html=True)
    st.markdown("Valores calculados basados en $\zeta$ y $\omega_n$.")

    st.markdown("<h5 style='color: #1E90FF;'>Sistema Mecánico (m-k-c)</h5>", unsafe_allow_html=True)
    st.code(f"Masa (m): {m:.2f} kg\nConstante Resorte (k): {k:.2f} N/m\nAmortiguamiento (c): {c:.2f} Ns/m", language="text")

    st.markdown("<h5 style='color: #1E90FF;'>Sistema Eléctrico (R-L-C)</h5>", unsafe_allow_html=True)
    st.code(f"Resistencia (R): {R_elec:.2f} Ω\nInductancia (L): {L_elec:.2f} H\nCapacitancia (C): {C_elec:.2f} F (asumida)", language="text")


with col2:
    st.markdown("<h3 style='color: #1E90FF;'>Parámetros Temporales (Lazo Cerrado)</h3>", unsafe_allow_html=True)
    st.markdown("Características clave de la respuesta al escalón.")

    # Lógica de visualización mejorada para cada caso
    if 0 < zeta < 1:
        st.code(
            f"Tiempo de levantamiento (Tr): {tr if isinstance(tr, str) else f'{tr:.3f} s'}\n"
            f"Sobreimpulso máximo (Mp): {mp:.2f} %\n"
            f"Tiempo de pico (Tp): {tp:.3f} s\n"
            f"Tiempo de establecimiento (Ts): {ts:.3f} s", language="text")
    elif zeta < 0:
        st.warning("Para un sistema inestable, los parámetros temporales no están definidos.")
    else: # ζ >= 1
        st.info("Para sistemas sobreamortiguados o críticamente amortiguados, no hay sobreimpulso ni tiempo de pico.")
        st.code(
            f"Tiempo de establecimiento (Ts): {ts if isinstance(ts, str) else f'{ts:.3f} s'}\n"
            f"Tiempo de levantamiento (Tr): {tr if isinstance(tr, str) else f'{tr:.3f} s'}", language="text")


st.markdown("---")

# --- Generación de Gráficos ---
tab1, tab2, tab3, tab4, tab5 = st.tabs(["Diagrama de Bode", "Polos y Ceros", "Respuesta al Impulso", "Respuesta al Escalón", "Respuesta a la Rampa"])

def plot_poles_zeros(system, ax, title):
    """Función para graficar polos y ceros."""
    poles = system.poles
    zeros = system.zeros
    ax.scatter(np.real(poles), np.imag(poles), marker='x', color='r', s=100, label='Polos')
    if zeros.size > 0:
        ax.scatter(np.real(zeros), np.imag(zeros), marker='o', color='b', s=100, facecolors='none', label='Ceros')
    ax.grid(True)
    ax.set_xlabel("Eje Real")
    ax.set_ylabel("Eje Imaginario")
    ax.axhline(0, color='black', lw=0.5)
    ax.axvline(0, color='black', lw=0.5)
    ax.set_title(title)
    ax.legend()

# Tab 1: Bode
with tab1:
    st.markdown("<h3 style='color: #1E90FF;'>Diagrama de Bode</h3>", unsafe_allow_html=True)
    fig, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(10, 8))
    w_ol, mag_ol, phase_ol = signal.bode(sys_ol)
    w_cl, mag_cl, phase_cl = signal.bode(sys_cl)
    ax_mag.semilogx(w_ol, mag_ol, label='Lazo Abierto')
    ax_mag.semilogx(w_cl, mag_cl, label='Lazo Cerrado', linestyle='--')
    ax_mag.set_ylabel("Magnitud (dB)")
    ax_mag.set_title("Respuesta en Magnitud")
    ax_mag.grid(True, which='both')
    ax_mag.legend()
    ax_phase.semilogx(w_ol, phase_ol, label='Lazo Abierto')
    ax_phase.semilogx(w_cl, phase_cl, label='Lazo Cerrado', linestyle='--')
    ax_phase.set_ylabel("Fase (grados)")
    ax_phase.set_xlabel("Frecuencia (rad/s)")
    ax_phase.set_title("Respuesta en Fase")
    ax_phase.grid(True, which='both')
    ax_phase.legend()
    plt.tight_layout()
    st.pyplot(fig)

# Tab 2: Polos y Ceros
with tab2:
    st.markdown("<h3 style='color: #1E90FF;'>Diagrama de Polos y Ceros</h3>", unsafe_allow_html=True)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    plot_poles_zeros(sys_ol, ax1, "Lazo Abierto")
    plot_poles_zeros(sys_cl, ax2, "Lazo Cerrado")
    st.pyplot(fig)

# Tab 3: Respuesta al Impulso
with tab3:
    st.markdown("<h3 style='color: #1E90FF;'>Respuesta al Impulso</h3>", unsafe_allow_html=True)
    t, y_ol = signal.impulse(sys_ol)
    _, y_cl = signal.impulse(sys_cl, T=t)
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(t, y_ol, label='Lazo Abierto')
    ax.plot(t, y_cl, label='Lazo Cerrado', linestyle='--')
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.set_title("Respuesta al Impulso del Sistema")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

# Tab 4: Respuesta al Escalón
with tab4:
    st.markdown("<h3 style='color: #1E90FF;'>Respuesta al Escalón Unitario</h3>", unsafe_allow_html=True)
    t, y_ol = signal.step(sys_ol)
    _, y_cl = signal.step(sys_cl, T=t)
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(t, y_ol, label='Lazo Abierto')
    ax.plot(t, y_cl, label='Lazo Cerrado', linestyle='--')
    ax.axhline(1, color='gray', linestyle=':', label='Referencia (Lazo Cerrado)')
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.set_title("Respuesta al Escalón del Sistema")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

# Tab 5: Respuesta a la Rampa
with tab5:
    st.markdown("<h3 style='color: #1E90FF;'>Respuesta a la Rampa</h3>", unsafe_allow_html=True)
    # Para simular rampa, se multiplica por 1/s y se obtiene la respuesta al escalón
    sys_ol_ramp = signal.TransferFunction(sys_ol.num, np.polymul(sys_ol.den, [1, 0]))
    sys_cl_ramp = signal.TransferFunction(sys_cl.num, np.polymul(sys_cl.den, [1, 0]))
    t, y_ol = signal.step(sys_ol_ramp)
    _, y_cl = signal.step(sys_cl_ramp, T=t)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(t, t, 'k:', label='Entrada Rampa')
    ax.plot(t, y_ol, label='Respuesta Lazo Abierto')
    ax.plot(t, y_cl, label='Respuesta Lazo Cerrado', linestyle='--')
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.set_title("Respuesta a la Rampa del Sistema")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

Writing 2_Punto_2.py


In [8]:
!mv 2_Punto_2.py pages/

#**Inicialización del Dashboard a partir de túnel local**
1. **Reemplazar nombre de archivo:** Reemplaza el nombre del archivo como se indica en el comentario de la linea 6 de la celda de codigo

2. **Accede al enlace provisional:** Una vez que la aplicación esté corriendo, LocalTunnel generará un enlace temporal. Haz clic o copia ese enlace para acceder a tu aplicación en el navegador (cada vez que corras la celda, el link podrá ser diferente).

**Nota:** Para finalizar la ejecución del Dashboard ejecuta la ultima celda de codigo y sigue las instrucciones.

In [9]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_👋_Hello.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-12-03 05:55:29--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-03 05:55:29--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-03T06%3A48%3A25Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-03